In [1]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


# Install Required Packages

In [2]:
!pip install -q accelerate==0.21.0 peft==0.4.0 bitsandbytes==0.40.2 transformers==4.31.0 trl==0.4.7 evaluate sacrebleu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.5/92.5 MB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 64.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.4/77.4 kB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.7/106.7 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 57.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 43.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 17.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 21.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 19.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━

# Import Required Libraries

In [3]:
import os
import torch
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import LoraConfig, PeftModel
from trl import SFTTrainer
import random
from tqdm import tqdm
import evaluate
import re

# Specify Our Training Parameters

In [4]:
model_name = "NousResearch/Llama-2-7b-chat-hf"

new_model = "Llama-2-7b-chat-finetune"

device_map = {"": 0}

# Interfacing with chatbot

In [5]:
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    AutoModel,
    pipeline,
    logging,
)

In [6]:
# path = '/content/drive/MyDrive/Graduation-Project/models/chatbot/' + new_model
# tokenizer = AutoTokenizer.from_pretrained(path, local_files_only=True)
# model = AutoModelForCausalLM.from_pretrained(path, load_in_4bit = True, local_files_only=True)

# Reload model in FP16 and merge it with LoRA weights
base_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    low_cpu_mem_usage=True,
    return_dict=True,
    torch_dtype=torch.float16,
    device_map=device_map,
)
model = PeftModel.from_pretrained(base_model, '/content/drive/MyDrive/Graduation-Project/models/chatbot/' + new_model)
model = model.merge_and_unload()

# Reload tokenizer to save it
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/583 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/746 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/21.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/435 [00:00<?, ?B/s]

In [7]:
# Ignore warnings
logging.set_verbosity(logging.CRITICAL)

# Run text generation pipeline with our next model
# pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=200)
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_new_tokens=256)

In [8]:
class chatbot():
  def __init__(self, pipe):
    self.history = ''
    self.instructions = 'you are a therapist, ask a question and be concise'
    self.pipe = pipe

  def get_prompt(self, input):
    return f"""<s>[INST]
        instructions: {self.instructions}
        conversation history:
        {self.history}
        input:
        {input} [/INST]"""

  def predict(self, input):
    # print(self.get_prompt(input))
    result = pipe(self.get_prompt(input))
    response = result[0]['generated_text'].split("[/INST]")[-1].strip()
    self.history += f'''
        User: {input}
        Ai: {response}
        '''

    return response


# interfacing with while loop

In [9]:
chat = chatbot(pipe)

while True:
    user_input = input("Prompt (press 'q' to quit): ")
    if user_input.lower() == 'q':
        break
    else:
        response = chat.predict(user_input)
        print("AI:", response)

Prompt (press 'q' to quit): what is llm


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1270: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation )
  warnings.warn(


AI: Great, thank you for sharing that with me. Can you tell me more about why you wanted to pursue an LLM degree? What are your career goals and how do you see this degree helping you achieve them?
Prompt (press 'q' to quit): i'm feeling sad can you help me out?
AI: Of course, I'm here to help. It sounds like you're going through a tough time. Can you tell me more about what's been going on and how you've been feeling? Sometimes talking about our emotions and experiences can help us process and understand them better.
Prompt (press 'q' to quit): q


# interfacing with gradio

In [ ]:
chat = chatbot(pipe)

# Set up the user interface
interface = gr.ChatInterface(
    clear_btn=None,
    fn=chat.predict,
    retry_btn=None,
    undo_btn=None,
)

# Launch the user interface
interface.launch(
    height=600,
    inline=True,
    share=True,
    width=800
)

In [ ]:
# Empty VRAM
del model
del pipe
import gc
gc.collect()
gc.collect()

20933

# thorwnnnn

In [ ]:
from langchain.chains import ConversationChain
from langchain.prompts.prompt import PromptTemplate
from langchain.memory import ConversationSummaryBufferMemory


In [ ]:
from langchain.llms import HuggingFacePipeline

llm = HuggingFacePipeline(pipeline=pipe)

In [ ]:
DEFAULT_TEMPLATE = """<s>[INST] <<SYS>>
you are acting as a therapist, ask a question and give a short replies
Current conversation:
{history}
<</SYS>>
{input} [/INST]"""
PROMPT = PromptTemplate(input_variables=["history", "input"], template=DEFAULT_TEMPLATE)

chain = ConversationChain(llm=llm, memory=ConversationSummaryBufferMemory(llm=llm, max_token_limit=20), prompt=PROMPT, verbose=True)

In [ ]:
import sys

In [ ]:
def predict(message: str, history: str):
    response = chain.predict(input=message)

    return response.split("[/INST]")[-1].strip()

In [ ]:
import gradio as gr

# Set up the user interface
interface = gr.ChatInterface(
    clear_btn=None,
    fn=predict,
    retry_btn=None,
    undo_btn=None,
)

# Launch the user interface
interface.launch(
    height=600,
    inline=True,
    share=True,
    width=800
)